In [ ]:
%load_ext autoreload
%autoreload 2

In [27]:
import jupyros
import rospy
import rospkg
from rospy_tutorials.srv import AddTwoInts, AddTwoIntsResponse
from turtlesim.srv import Spawn
from random import randint
from math import radians
from ipywidgets import Image
from ipycanvas import hold_canvas, Canvas

# Add Two Ints

In [3]:
rospy.init_node("adder")

In [4]:
def handle_add_two_ints(req):
    print("Returning [%s + %s = %s]"%(req.a, req.b, (req.a + req.b)))
    return AddTwoIntsResponse(req.a + req.b)

In [5]:
%%thread_cell

srv = rospy.Service('add_two_ints', AddTwoInts, handle_add_two_ints)

Output(layout=Layout(border='1px solid gray'))

In [6]:
jupyros.client('add_two_ints', AddTwoInts)

# Turtlesim

In [7]:
# !rosrun turtlesim turtlesim_node

In [ ]:
jupyros.client('spawn', Spawn)

In [70]:
def get_random_turtle():
    turtles_svg = ['hydro', 'indigo', 'kinetic', 'lunar']
    turtles_png = ['box-turtle', 'diamondback', 'electric', 'fuerte', 'groovy', 'hydro', 'indigo', 'jade',
                   'kinetic', 'lunar', 'melodic', 'noetic', 'robot-turtle', 'sea-turtle', 'turtle']
    
    # Find path to a random turtle image
    r = rospkg.RosPack()
    turtlesim_path = r.get_path('turtlesim')
    turtle_path = turtlesim_path + '/images/' + turtles_png[randint(0, len(turtles_png)-1)] + '.png'

    turtle_img = Image.from_file(turtle_path)

    return turtle_img


def spawn_turtle(spawn_canvas):
    turtle_img = get_random_turtle()
    
    # Center of canvas
    x = spawn_canvas.width // 2
    y = spawn_canvas.height // 2
    
    spawn_canvas.draw_image(turtle_img, x, y, width=100)

    return spawn_canvas

In [12]:
import rostopic

pubs, _ = rostopic.get_topic_list()
pubs

# Retrieve name of last turtle spawned
last_turtle = pubs[-1][0].split('/')[1]
print(last_turtle)

rosout


In [32]:
canvas = Canvas(width=1600, height=1200, layout=dict(width="100%"))
display(canvas)


# Water
canvas.fill_style = "#4556FF"
canvas.fill_rect(0, 0, canvas.width, canvas.height)

with hold_canvas(canvas):
    for _ in range(20):
        turtle_width = 90
        x_pos = randint(0, canvas.width-1)
        y_pos = randint(0, canvas.height-1)
        theta = radians(90)

        turtle_img = get_random_turtle()
        
        canvas.translate(x_pos, y_pos)
        canvas.rotate(-theta)

        canvas.draw_image(turtle_img, width=turtle_width)
        
        canvas.rotate(theta)
        canvas.translate(-x_pos, -y_pos)


Canvas(height=1200, layout=Layout(width='100%'), width=1600)

### Spawn turtle

In [69]:
canvas = Canvas(width=1600, height=1200, layout=dict(width="100%"))
display(canvas)

# Water
canvas.fill_style = "#4556FF"
canvas.fill_rect(0, 0, canvas.width, canvas.height)

canvas = spawn_turtle(canvas)

Canvas(height=1200, layout=Layout(width='100%'), width=1600)

In [24]:
from time import sleep

# Number of steps in your animation
steps_number = 200

offscreen_canvas = Canvas(width=100, height=100)
offscreen_canvas.draw_image(turtle_img, width=100)

for i in range(steps_number):
    with hold_canvas(canvas):
        # Clear the old animation step
        canvas.clear()

        # Perfom all your drawings here
        canvas.fill_style = '#4556FF'
        canvas.fill_rect(0, 0, canvas.width, canvas.height)

        # canvas.fill_style = 'green'
        # canvas.fill_circle(x_pos+i, y_pos-i, 30)
        canvas.draw_image(offscreen_canvas, x_pos+i, y_pos-i, width=turtle_width)

    # Animation frequency ~50Hz = 1000./50. milliseconds
    sleep(0.02)

In [14]:
from ipycanvas import hold_canvas, MultiCanvas

canvas = MultiCanvas(width=1600, height=1200, layout=dict(width="100%"))
display(canvas)


# Water
canvas[0].fill_style = "#4556FF"
canvas[0].fill_rect(0, 0, canvas.width, canvas.height)

turtle_width = 90
x_pos = canvas.width // 2 - turtle_width // 2
y_pos = canvas.height // 2 - turtle_width // 2
theta = 0
turtle_img = get_random_turtle()

canvas[1].draw_image(turtle_img, x=x_pos, y=y_pos, width=turtle_width)


MultiCanvas(height=1200, layout=Layout(width='100%'), width=1600)

In [ ]:
from geometry_msgs.msg import Twist

topic_name = '/turtle1/cmd_vel'

jupyros.publish(topic_name, Twist)

# Moving turtle animation

In [15]:
from time import sleep

# Number of steps in your animation
steps_number = 200

# Note how `hold_canvas` now wraps the entire for-loop
for i in range(steps_number):
    with hold_canvas(canvas):
        # Clear the old animation step
        canvas[1].clear()

        # Perfom all your drawings here
#         canvas.fill_rect(0, 0, canvas.width, canvas.height)
        canvas[1].draw_image(turtle_img, x_pos+i, y_pos-i, width=turtle_width)

        # Animation frequency ~50Hz = 1000./50. milliseconds
        sleep(.02)

AttributeError: 'MultiCanvas' object has no attribute 'fill_rect'